**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
! sudo pip install scikit-video

     |████████████████████████████████| 2.3MB 2.9MB/s 


In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Dropout
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

$$
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
$$

where: 
$$p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
$$

We note the $Q$-function:

$$Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
$$

Thus, the optimal Q function is:
$$
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
$$

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function '''act''' takes as input the current state s and a boolean variable Train.
If Train is set to True. Then the Agent will choose at this current state either a random action or the best action it has learned previously with the following probabilities.

1.   Probability of Exploration : $100*\epsilon \% $
2.   Probability of Exploitation: $100*(1-\epsilon) \%$

The '''epsilon''' is esseantial because it handles the trade-off "Exploration" "Exploitation". In other words, our agent should on one hand explore the space of actions in order to learn about them. On the other hand, our agent should be able to apply what it has learned previously to pich the action that will maximize the future reward.



***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position``` : is an array that represents the possible cells where the rat can go. It contains zeros everywhere, except for the boundaries, where the rat can't go, that has -1 as a value and also the position of the rat where the zero is replaced by one.

```board``` : is an array that represent the reward of each cell (0.5 if the cell contains cheese, -1 if poison). If the rat goes through a cell, the value change to zero.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0,self.n_action,size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 7.5/16.0. Average score (-8.5)
Win/lose count 7.5/11.0. Average score (-6.0)
Win/lose count 7.5/17.0. Average score (-7.166666666666667)
Win/lose count 6.5/10.0. Average score (-6.25)
Win/lose count 13.0/16.0. Average score (-5.6)
Win/lose count 6.5/16.0. Average score (-6.25)
Win/lose count 8.5/18.0. Average score (-6.714285714285714)
Win/lose count 9.5/15.0. Average score (-6.5625)
Win/lose count 11.0/14.0. Average score (-6.166666666666667)
Win/lose count 10.0/13.0. Average score (-5.85)
Win/lose count 10.5/15.0. Average score (-5.7272727272727275)
Win/lose count 8.5/8.0. Average score (-5.208333333333333)
Win/lose count 12.5/13.0. Average score (-4.846153846153846)
Win/lose count 8.0/14.0. Average score (-4.928571428571429)
Win/lose count 14.0/19.0. Average score (-4.933333333333334)
Win/lose count 11.0/14.0. Average score (-4.8125)
Win/lose count 13.5/7.0. Average score (-4.147058823529412)
Win/lose count 8.5/17.0. Average score (-4.388888888888889)
Win/lose count 1

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

$$
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
$$

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

$$
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
$$
Finally, deduce that a plausible objective is:

$$
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
$$




*   Let $\pi$ be a policy:

We Have $$Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
$$
That can be written as follow,
$$
Q^\pi(s,a)=E_{p^{\pi}}[r(s,a) + \sum_{0<t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
$$
$$
Q^\pi(s,a)=E_{p^{\pi}}[r(s,a)|s_{0}=s,a_{0}=a] \>  + E_{p^{\pi}}[\sum_{0<t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
$$
$$
Q^\pi(s,a)=r(s,a)  + \gamma*E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
$$
$$
Q^\pi(s,a)=r(s,a)  +\gamma* E_{(s',a')\sim p(.|s,a)}[ Q^{\pi}(s',a')]
$$
Finally we get,
$$
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
$$


\\
\\


*      We have:

$$ Q^{*}(s,a) = \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$

$$ Q^{*}(s,a) = \max_{\pi} r(s, a) + \gamma \sum p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi}(s',a') $$

$$ Q^{*}(s,a) = r(s,a) + \gamma \max_{\pi} \sum p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \qquad \text{with } \pi=(a',\pi') $$

We can exchange the maximum and the sum with the following inequality :

$$ \max_{\pi} \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \leq \sum_{(s',a')} p(s_{t+1}=s' |s_t = s, a_t = a ) \max_{\pi}Q^{\pi'}(s',a') $$

Then we have : (considering $\hat{\pi}(s',a') = \arg\max_{\pi'} Q^{\pi'}(s',a')$)

$$
\begin{align*}
 \sum_{s'} p(s_{t+1}=s'|s_t = s, a_t = a ) \max_{\pi'}Q^{\pi'}(s',a') &= \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a ) Q^{\hat{\pi}(s',a')}(s',a') \\
 &\leq \max_{\pi'} \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') 
\end{align*}
$$

So :

$$ Q^{*}(s,a) = r(s,a) + \gamma  \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'}\max_{\pi'}Q^{\pi'}(s',a') $$


$$ Q^{*}(s,a) = r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'} Q^{*}(s',a') $$

Finally:$$ Q^{*}(s, a) = E_{s' \sim \pi^{*}(\cdot|s,a)} \left[ r(s, a) + \underset{a'}{\max} \gamma Q^{*}(s', a') \right]$$

*   Let $Q^{\theta}(s, a) = Q(s, a, \theta)$ be our $Q$ function (approx. by a neural network). We want $Q^{\theta} \approx Q^{*}$. In other terms, $Q^{\theta}$ should verify :

$$ Q^{\theta}(s, a) = E_{s'\sim \pi^*(.|s,a)} \left[ r(s,a)+\gamma\max_{a'}Q^{\theta}(s',a') \right] $$

$$ E_{s'\sim \pi^*(.|s,a)} \left[ r(s,a)+\gamma\max_{a'}Q^{\theta}(s',a') - Q^{\theta}(s, a) \right] = 0 $$

$$ \left|\left| E_{s'\sim \pi^*(.|s,a)} \left[ r(s,a)+\gamma\max_{a'}Q^{\theta}(s',a') - Q^{\theta}(s, a) \right] \right|\right|^2 = 0 $$

Hence :

$$ E_{s'\sim \pi^*(.|s,a)} \left|\left| r(s,a)+\gamma\max_{a'}Q^{\theta}(s',a') - Q^{\theta}(s, a) \right|\right|^2 = 0 $$

An objective function can be then defined as:

$$ \mathcal{L}(\theta) = E_{s'\sim \pi^*(.|s,a)} \left|\left| r + \gamma\max_{a'}Q(s', a', \theta) - Q(s, a, \theta) \right|\right|^2 $$

that has to be minimized.



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
          self.memory=self.memory[1:]
        

    def random_access(self):
        return self.memory[np.random.randint(0,len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s]))[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i]=s_
            target_q[i]=self.model.predict(np.array([s_]))
            if game_over_:
                target_q[i,a_]=r_
            else:
                target_q[i,a_]=r_+self.discount*np.max(self.model.predict(np.array([n_s_])))
                ######## FILL IN
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

class DQN_FC(DQN):
    def __init__(self, lr=0.1, *args, **kwargs):
        super(DQN_FC, self).__init__(*args, **kwargs)
        
        # NN Model

        input_model = keras.layers.Input(shape=(5, 5, self.n_state), name="input")
        flatten = keras.layers.Flatten(name="flatten")(input_model)
        fc1 = Dense(100, name="fc1", activation="relu")(flatten)
        fc2 = Dense(100, name="fc2", activation="relu")(fc1)
        fc3 = Dense(100, name="fc3", activation="relu")(fc2)
        output_model = Dense(4, name="output")(fc3)
        model = keras.models.Model(input_model, output_model)
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model            
# class DQN_FC(DQN):
#     def __init__(self, *args, lr=0.1,**kwargs):
#         super(DQN_FC, self).__init__( *args,**kwargs)
        
#         # NN Model
        
#         ####### FILL IN
        
#         model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
#         self.model = model
        

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(lr=.1, grid_size=size, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))


Epoch 000/100 | Loss 0.0113 | Win/lose count 10.5/16.0 (-5.5)
Epoch 001/100 | Loss 0.0050 | Win/lose count 5.0/2.0 (3.0)
Epoch 002/100 | Loss 0.0028 | Win/lose count 2.5/0 (2.5)
Epoch 003/100 | Loss 0.0080 | Win/lose count 1.5/3.0 (-1.5)
Epoch 004/100 | Loss 0.0033 | Win/lose count 6.5/4.0 (2.5)
Epoch 005/100 | Loss 0.0095 | Win/lose count 3.5/4.0 (-0.5)
Epoch 006/100 | Loss 0.1012 | Win/lose count 13.0/6.0 (7.0)
Epoch 007/100 | Loss 0.0045 | Win/lose count 9.0/2.0 (7.0)
Epoch 008/100 | Loss 0.0063 | Win/lose count 5.0/2.0 (3.0)
Epoch 009/100 | Loss 0.0612 | Win/lose count 3.5/5.0 (-1.5)
Epoch 010/100 | Loss 0.0189 | Win/lose count 10.0/5.0 (5.0)
Epoch 011/100 | Loss 0.0034 | Win/lose count 8.0/5.0 (3.0)
Epoch 012/100 | Loss 0.0077 | Win/lose count 1.5/1.0 (0.5)
Epoch 013/100 | Loss 0.0043 | Win/lose count 5.0/2.0 (3.0)
Epoch 014/100 | Loss 0.0100 | Win/lose count 5.5/4.0 (1.5)
Epoch 015/100 | Loss 0.0022 | Win/lose count 3.0/1.0 (2.0)
Epoch 016/100 | Loss 0.0112 | Win/lose count 10.5/

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_fc = DQN_FC( lr=.1,grid_size=size, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')


Test of the FC
Win/lose count 3.5/4.0. Average score (-0.5)
Win/lose count 2.0/1.0. Average score (0.25)
Win/lose count 2.0/5.0. Average score (-0.8333333333333334)
Win/lose count 0.5/0. Average score (-0.5)
Win/lose count 3.5/2.0. Average score (-0.1)
Win/lose count 2.0/1.0. Average score (0.08333333333333333)
Win/lose count 6.0/8.0. Average score (-0.21428571428571427)
Win/lose count 2.0/4.0. Average score (-0.4375)
Win/lose count 3.5/7.0. Average score (-0.7777777777777778)
Win/lose count 5.0/8.0. Average score (-1.0)
Win/lose count 2.0/7.0. Average score (-1.3636363636363635)
Win/lose count 3.0/6.0. Average score (-1.5)
Win/lose count 2.5/3.0. Average score (-1.4230769230769231)
Win/lose count 5.0/7.0. Average score (-1.4642857142857142)
Win/lose count 7.0/11.0. Average score (-1.6333333333333333)
Win/lose count 3.5/6.0. Average score (-1.6875)
Win/lose count 2.0/7.0. Average score (-1.8823529411764706)
Win/lose count 2.5/5.0. Average score (-1.9166666666666667)
Win/lose count 7.5/

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, lr=0.1, *args, **kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        input_model = keras.layers.Input(shape=(5, 5, self.n_state), name="input")
        cn1_1 = Conv2D(32, 3, padding="same", activation="relu", name="cn1_1")(input_model)
        cn1_2 = Conv2D(32, 3, padding="same", activation="relu", name="cn1_2")(cn1_1)
        mp1 = MaxPooling2D(name="mp1")(cn1_2)
        cn2 = Conv2D(64, 3, padding="same", activation="relu", name="cn2")(mp1)
        flatten = keras.layers.Flatten(name="flatten")(cn2)
        fc1= Dense(120, name="fc1", activation="relu")(flatten)
        output_model = Dense(4, name="output")(fc1)
        model = keras.models.Model(input_model, output_model)
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.1, grid_size=size, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/100 | Loss 0.0056 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/100 | Loss 0.0002 | Win/lose count 2.5/3.0 (-0.5)
Epoch 002/100 | Loss 0.0029 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/100 | Loss 0.0563 | Win/lose count 2.5/4.0 (-1.5)
Epoch 004/100 | Loss 0.0020 | Win/lose count 2.5/5.0 (-2.5)
Epoch 005/100 | Loss 0.0091 | Win/lose count 4.0/2.0 (2.0)
Epoch 006/100 | Loss 0.0125 | Win/lose count 5.5/3.0 (2.5)
Epoch 007/100 | Loss 0.0016 | Win/lose count 1.5/5.0 (-3.5)
Epoch 008/100 | Loss 0.0060 | Win/lose count 3.0/6.0 (-3.0)
Epoch 009/100 | Loss 0.0025 | Win/lose count 3.5/6.0 (-2.5)
Epoch 010/100 | Loss 0.0563 | Win/lose count 3.0/3.0 (0.0)
Epoch 011/100 | Loss 0.0034 | Win/lose count 3.5/3.0 (0.5)
Epoch 012/100 | Loss 0.0069 | Win/lose count 2.5/4.0 (-1.5)
Epoch 013/100 | Loss 0.0140 | Win/lose count 1.0/1.0 (0.0)
Epoch 014/100 | Loss 0.0026 | Win/lose count 2.5/4.0 (-1.5)
Epoch 015/100 | Loss 0.0051 | Win/lose count 6.0/0 (6.0)
Epoch 016/100 | Loss 0.0029 | Win/lose count 6.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN( lr=.1,grid_size=size,epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')

Test of the CNN
Win/lose count 5.0/2.0. Average score (3.0)
Win/lose count 10.0/3.0. Average score (5.0)
Win/lose count 8.0/2.0. Average score (5.333333333333333)
Win/lose count 7.0/2.0. Average score (5.25)
Win/lose count 5.0/3.0. Average score (4.6)
Win/lose count 3.5/0. Average score (4.416666666666667)
Win/lose count 7.5/1.0. Average score (4.714285714285714)
Win/lose count 10.0/1.0. Average score (5.25)
Win/lose count 8.0/1.0. Average score (5.444444444444445)
Win/lose count 10.0/2.0. Average score (5.7)
Win/lose count 7.0/2.0. Average score (5.636363636363637)
Win/lose count 1.0/2.0. Average score (5.083333333333333)
Win/lose count 3.0/1.0. Average score (4.846153846153846)
Win/lose count 6.5/3.0. Average score (4.75)
Win/lose count 6.5/0. Average score (4.866666666666666)
Win/lose count 19.5/3.0. Average score (5.59375)
Win/lose count 10.0/3.0. Average score (5.676470588235294)
Win/lose count 3.5/1.0. Average score (5.5)
Win/lose count 3.0/3.0. Average score (5.2105263157894735)

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.8)

In [0]:
print('Test of the FC')
test(agent_fc, env, epochs_test, prefix='fc_test')
print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='cnn_test')

Test of the FC
Win/lose count 22.0/5.0. Average score (17.0)
Win/lose count 8.0/6.0. Average score (9.5)
Win/lose count 10.0/5.0. Average score (8.0)
Win/lose count 11.0/6.0. Average score (7.25)
Win/lose count 11.0/6.0. Average score (6.8)
Win/lose count 18.5/12.0. Average score (6.75)
Win/lose count 9.0/8.0. Average score (5.928571428571429)
Win/lose count 11.0/6.0. Average score (5.8125)
Win/lose count 9.0/5.0. Average score (5.611111111111111)
Win/lose count 9.0/1.0. Average score (5.85)
Win/lose count 9.0/2.0. Average score (5.954545454545454)
Win/lose count 10.5/3.0. Average score (6.083333333333333)
Win/lose count 10.5/3.0. Average score (6.1923076923076925)
Win/lose count 9.0/4.0. Average score (6.107142857142857)
Win/lose count 12.5/6.0. Average score (6.133333333333334)
Win/lose count 7.0/3.0. Average score (6.0)
Win/lose count 22.0/8.0. Average score (6.470588235294118)
Win/lose count 2.5/2.0. Average score (6.138888888888889)
Win/lose count 7.5/4.0. Average score (6.0)
Win/

In [0]:
HTML(display_videos('fc_test10.mp4'))

In [0]:
HTML(display_videos('cnn_test10.mp4'))

We can observe that one main issue is that the agent keeps alternating between two states (2 positions).

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
# def train_explore(agent,env,epoch,prefix=''):
#     pass
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    decreasing_factor=0.9

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action,train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        agent.set_epsilon(agent.epsilon * decreasing_factor)
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json') 

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train='False'):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))
        self.malus_position[self.x, self.y] = 0.1

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        if train:
          reward= self.board[self.x, self.y]-self.malus_position[self.x, self.y]
        else: 
          reward=0
        # reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        # self.malus_position[self.x, self.y] = 0.1


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state       

In [0]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.1,grid_size= size, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0093 | Win/lose count 5.5/25.500000000000057 (-20.000000000000057)
Epoch 001/100 | Loss 0.0286 | Win/lose count 7.5/28.800000000000043 (-21.300000000000043)
Epoch 002/100 | Loss 0.0074 | Win/lose count 7.0/19.70000000000002 (-12.70000000000002)
Epoch 003/100 | Loss 0.0143 | Win/lose count 5.0/20.50000000000003 (-15.500000000000028)
Epoch 004/100 | Loss 0.0113 | Win/lose count 10.5/16.699999999999974 (-6.199999999999974)
Epoch 005/100 | Loss 0.0120 | Win/lose count 12.0/16.89999999999997 (-4.89999999999997)
Epoch 006/100 | Loss 0.0068 | Win/lose count 4.0/17.899999999999984 (-13.899999999999984)
Epoch 007/100 | Loss 0.0087 | Win/lose count 13.0/16.19999999999996 (-3.19999999999996)
Epoch 008/100 | Loss 0.0127 | Win/lose count 2.0/19.200000000000003 (-17.200000000000003)
Epoch 009/100 | Loss 0.0158 | Win/lose count 3.5/19.39999999999999 (-15.899999999999991)
Epoch 010/100 | Loss 0.0079 | Win/lose count 10.0/15.69999999999996 (-5.69999999999996)
Epoch 011/100 | Loss

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 16.0/13.09999999999997. Average score (2.9000000000000306)
Win/lose count 14.0/14.599999999999964. Average score (1.1500000000000332)
Win/lose count 13.0/15.199999999999962. Average score (0.03333333333336815)
Win/lose count 20.0/15.899999999999963. Average score (1.0500000000000353)
Win/lose count 22.5/11.89999999999998. Average score (2.960000000000032)
Win/lose count 0/20.000000000000014. Average score (-0.8666666666666423)
Win/lose count 19.5/16.999999999999986. Average score (-0.38571428571426275)
Win/lose count 26.5/13.99999999999997. Average score (1.2250000000000238)
Win/lose count 8.0/17.499999999999982. Average score (0.03333333333335631)
Win/lose count 23.0/15.399999999999963. Average score (0.7900000000000243)
Win/lose count 14.0/14.599999999999964. Average score (0.663636363636389)
Win/lose count 24.5/12.199999999999973. Average score (1.6333333333333588)
Win/lose count 19.5/14.999999999999963. Average score (1.8538461538461801)
Win/lose count 9.5/15.5999999

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***